**Install python dependencies**

In [ ]:
!pip install -q -r ./dependencies/requirements.txt

**Load python libraries**

In [2]:
import pandas as pd
from sklearn import tree

**Load Data**

In [10]:
inputs = pd.read_csv('./data/inputs.csv')
label = pd.read_csv('./data/label.csv')

**Drop Bettings Odds and Position**

In [11]:
inputs = inputs.drop(['odds_win', 'odds_draw', 'odds_lose', 'position_normalized'], axis='columns')

**Create Deicision Tree**

In [13]:
model = tree.DecisionTreeRegressor()
model = model.fit(inputs, label)

model.score(inputs, label)

1.0